# Stock Modeling using Gemini Sentiments

In [1]:
# import libraries
import os
from google.colab import userdata
from google.colab import output
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

In [2]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

fatal: destination path 'Financial_Sentiment_LLM' already exists and is not an empty directory.


In [3]:
# Read in outputs from Gemini Sentiment Analysis
gemini_prompt1_sentiment = pd.read_csv('/content/Financial_Sentiment_LLM/03_Sentiment_Analysis/Prompt1_Sentiment_Analysis_Results.csv')
gemini_prompt1_sentiment = gemini_prompt1_sentiment.fillna('')
print(gemini_prompt1_sentiment.shape)
display(gemini_prompt1_sentiment.head())

(10125, 11)


,Source,Unique_ID,Ticker,Date,URL,Finance,Production,Reserves / Exploration / Acquisitions / Mergers / Divestments,Environment / Regulatory / Geopolitics,Alternative Energy / Lower Carbon,Oil Price / Natural Gas Price / Gasoline Price
0,Investment Research,IR-1,MRO,5/16/2024,,Positive,Neutral,Positive,Negative,Neutral,Neutral
1,Investment Research,IR-2,EOG,5/14/2024,,Positive,Neutral,Positive,Neutral,Positive,Neutral
2,Investment Research,IR-3,EOG,5/11/2024,,Positive,Positive,Positive,Neutral,Neutral,Neutral
3,Investment Research,IR-4,DVN,5/11/2024,,Positive,Neutral,Positive,Negative,Neutral,Neutral
4,Investment Research,IR-5,COP,5/7/2024,,Positive,Neutral,Neutral,Negative,Neutral,Neutral


In [4]:
# Define a list of date formats
date_formats = ['%d/%m/%Y',  # 01/01/2021
                '%m/%d/%Y',  # 5/16/2024
                '%Y-%m-%d',  # 2021-02-28
                '%B %d, %Y',  # March 15, 2021
                '%b-%d-%Y',  # Apr-30-2019
                '%Y.%m.%d',  # 2021.04.10
                '%m-%d-%Y',  # 04-10-2021
                '%d-%b-%y',  # 17-Aug-22
                '%b-%y']  # Sep-19

# Function to parse dates with multiple formats
def parse_date(date_str):
    """
    Parse a date string using a list of possible date formats.

    This function attempts to parse a given date string using various date formats specified in `date_formats`.
    If the date string matches a format that only includes the month and year ('%b-%y'), it defaults the day to the first day of the month.

    Args:
        date_str (str): The date string to parse.

    Returns:
        datetime: The parsed date as a `datetime` object.

    Raises:
        ValueError: If the date string does not match any of the provided formats in `date_formats`.
    """
    for fmt in date_formats:
        try:
            # Handle special case where only month and year are given
            if fmt == '%b-%y':
                parsed_date = datetime.strptime(date_str, fmt)
                # Default to the first day of the month
                return parsed_date.replace(day=1)
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    raise ValueError(f"Date format not recognized: {date_str}")

# Apply the function to the 'Date' column
gemini_prompt1_sentiment['Date'] = gemini_prompt1_sentiment['Date'].apply(parse_date)

# Format the dates in a specific string format and convert to datetime
gemini_prompt1_sentiment['Date'] = gemini_prompt1_sentiment['Date'].dt.strftime('%Y-%m-%d')
gemini_prompt1_sentiment['Date'] = pd.to_datetime(gemini_prompt1_sentiment['Date'])

# Display the DataFrame
print(gemini_prompt1_sentiment.shape)
print(gemini_prompt1_sentiment['Date'].dtype)
display(gemini_prompt1_sentiment.head())
print(gemini_prompt1_sentiment.isna().sum())

(10125, 11)
datetime64[ns]


,Source,Unique_ID,Ticker,Date,URL,Finance,Production,Reserves / Exploration / Acquisitions / Mergers / Divestments,Environment / Regulatory / Geopolitics,Alternative Energy / Lower Carbon,Oil Price / Natural Gas Price / Gasoline Price
0,Investment Research,IR-1,MRO,2024-05-16,,Positive,Neutral,Positive,Negative,Neutral,Neutral
1,Investment Research,IR-2,EOG,2024-05-14,,Positive,Neutral,Positive,Neutral,Positive,Neutral
2,Investment Research,IR-3,EOG,2024-11-05,,Positive,Positive,Positive,Neutral,Neutral,Neutral
3,Investment Research,IR-4,DVN,2024-11-05,,Positive,Neutral,Positive,Negative,Neutral,Neutral
4,Investment Research,IR-5,COP,2024-07-05,,Positive,Neutral,Neutral,Negative,Neutral,Neutral


Source                                                           0
Unique_ID                                                        0
Ticker                                                           0
Date                                                             0
URL                                                              0
Finance                                                          0
Production                                                       0
Reserves / Exploration / Acquisitions / Mergers / Divestments    0
Environment / Regulatory / Geopolitics                           0
Alternative Energy / Lower Carbon                                0
Oil Price / Natural Gas Price / Gasoline Price                   0
dtype: int64


In [5]:
# Renaming the columns to simplify sentiment categories
gemini_prompt1_sentiment.rename(columns={
    'Reserves / Exploration / Acquisitions / Mergers / Divestments': 'Reserves/M&A',
    'Environment / Regulatory / Geopolitics': 'Regulatory',
    'Alternative Energy / Lower Carbon': 'Green Energy',
    'Oil Price / Natural Gas Price / Gasoline Price': 'OG Price'}
                                , inplace=True)

display(gemini_prompt1_sentiment.head())

,Source,Unique_ID,Ticker,Date,URL,Finance,Production,Reserves/M&A,Regulatory,Green Energy,OG Price
0,Investment Research,IR-1,MRO,2024-05-16,,Positive,Neutral,Positive,Negative,Neutral,Neutral
1,Investment Research,IR-2,EOG,2024-05-14,,Positive,Neutral,Positive,Neutral,Positive,Neutral
2,Investment Research,IR-3,EOG,2024-11-05,,Positive,Positive,Positive,Neutral,Neutral,Neutral
3,Investment Research,IR-4,DVN,2024-11-05,,Positive,Neutral,Positive,Negative,Neutral,Neutral
4,Investment Research,IR-5,COP,2024-07-05,,Positive,Neutral,Neutral,Negative,Neutral,Neutral


In [6]:
#Identify minimum article date for stock price download
min_date = gemini_prompt1_sentiment['Date'].min()
print(min_date)

2019-01-02 00:00:00


In [7]:
ticker_list = gemini_prompt1_sentiment['Ticker'].unique().tolist()
print(ticker_list)

['MRO', 'EOG', 'DVN', 'COP', 'PXD', 'PDCE', 'CXO', 'CVX', 'MPC', 'HES', 'PSX', 'XOM', 'SHEL', 'BP', 'OXY', 'VLO', 'TTE', 'EQNR']


In [8]:
#Pull YahooFinance stock data to get Returns (ret)
all_data = {}

for ticker in ticker_list:
    try:
        # Get the last date for the current ticker from gemini_prompt1_sentiment
        last_date = gemini_prompt1_sentiment[gemini_prompt1_sentiment['Ticker'] == ticker]['Date'].max()
        if pd.isna(last_date):
            print(f"No data available for {ticker} in gemini_prompt1_sentiment.")
            continue

        # Ensure the date is in the correct format
        last_date = pd.to_datetime(last_date).strftime('%Y-%m-%d')
        print(f"Downloading data for {ticker} from {min_date} to {last_date}")

        # Download stock data
        data = yf.download(ticker, start=min_date, end=last_date)["Adj Close"]

        if data.empty:
            print(f"No data retrieved for {ticker}")
        else:
            all_data[ticker] = data

    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")

# Combine all data into a single DataFrame
if all_data:
    price_df = pd.DataFrame(all_data)
    ret_df = price_df.pct_change().dropna()
    ret_df.name = "ret"
    ret_df.describe()
else:
    print("No data available.")

print()
print(ret_df.shape)
print()
display(ret_df.head())
print()
display(ret_df.describe())

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['PDCE']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


...


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed



(1381, 16)



,MRO,EOG,DVN,COP,PXD,CVX,MPC,HES,PSX,XOM,SHEL,BP,OXY,VLO,TTE,EQNR
Date,,,,,,,,,,,,,,,,
2019-01-03,-0.006794,-0.007063,0.013356,-0.018934,0.003202,-0.019153,-0.014588,0.003551,-0.013845,-0.015353,-0.002191,0.005701,-0.012258,-0.017999,0.002273,0.002313
2019-01-04,0.054035,0.043903,0.044643,0.025249,0.033846,0.020725,0.049005,0.077848,0.045339,0.036870,0.022631,0.031435,0.032169,0.037870,0.029295,0.028611
2019-01-07,0.013628,0.006282,0.031746,-0.004392,0.021753,0.012994,0.009894,0.047932,0.006936,0.005200,0.001321,0.003247,0.020092,0.001039,-0.001836,0.008524
2019-01-08,-0.008963,0.010687,0.007889,0.013392,-0.000843,-0.004365,0.009798,0.012531,-0.000875,0.007271,-0.006103,-0.003237,0.007909,0.003502,-0.003127,0.004449
2019-01-09,0.038114,0.038421,0.010959,0.033116,0.025387,0.013420,0.021473,0.034654,0.006565,0.005275,0.011616,0.008244,0.012156,-0.000905,0.021960,0.015057


,MRO,EOG,DVN,COP,PXD,CVX,MPC,HES,PSX,XOM,SHEL,BP,OXY,VLO,TTE,EQNR
count,1381.000000,1381.000000,1381.000000,1381.000000,1381.000000,1381.000000,1381.000000,1381.000000,1381.000000,1381.000000,1381.000000,1381.000000,1381.000000,1381.000000,1381.000000,1381.000000
mean,0.001214,0.000845,0.001394,0.000937,0.001114,0.000660,0.001339,0.001435,0.000839,0.000779,0.000569,0.000430,0.000877,0.001136,0.000643,0.000726
std,0.035932,0.029290,0.036199,0.027140,0.028747,0.021593,0.029042,0.030401,0.025785,0.020884,0.022506,0.022826,0.038225,0.029546,0.021002,0.023721
min,-0.468521,-0.320072,-0.373972,-0.248400,-0.369197,-0.221248,-0.270089,-0.336686,-0.158658,-0.122248,-0.171722,-0.191040,-0.520138,-0.192209,-0.178208,-0.210562
25%,-0.016748,-0.013923,-0.016544,-0.011790,-0.013137,-0.008451,-0.011963,-0.014134,-0.010586,-0.010183,-0.009659,-0.009951,-0.014843,-0.012679,-0.009734,-0.011118
50%,0.000704,0.000000,0.000826,0.000000,0.000049,0.000739,0.001808,0.001129,0.000825,0.000481,0.000583,0.000000,0.000000,0.000511,0.001372,0.000000
75%,0.017346,0.014470,0.017629,0.014067,0.013902,0.009532,0.014867,0.015287,0.013581,0.011223,0.010660,0.009766,0.015107,0.015804,0.010430,0.013315
max,0.232446,0.165703,0.210721,0.252138,0.204343,0.227407,0.206286,0.203154,0.221722,0.126868,0.196795,0.216053,0.336977,0.312025,0.152756,0.133043


In [9]:
# reshape ret same day
ret_stack = ret_df.stack().reset_index()
ret_stack.columns = ['Date', 'Ticker', 'Returns']
ret_stack['Date'] = pd.to_datetime(ret_stack['Date'])
print(ret_stack['Date'].dtype)
display(ret_stack.head())

datetime64[ns]


,Date,Ticker,Returns
0,2019-01-03,MRO,-0.006794
1,2019-01-03,EOG,-0.007063
2,2019-01-03,DVN,0.013356
3,2019-01-03,COP,-0.018934
4,2019-01-03,PXD,0.003202


In [10]:
# merge in returns for the day from YahooFinance
df_merged = pd.merge(gemini_prompt1_sentiment, ret_stack, on=['Ticker', 'Date'], how='left')
df_merged = df_merged.sort_values(['Ticker', 'Date'])
display(df_merged.head())
print(df_merged.isna().sum())

,Source,Unique_ID,Ticker,Date,URL,Finance,Production,Reserves/M&A,Regulatory,Green Energy,OG Price,Returns
9863,Earnings Call Presentations,EP-331,BP,2019-02-05,,Positive,Positive,Positive,Positive,Positive,Neutral,0.034550
10113,Earnings Call Q&A,EQ-331,BP,2019-02-05,,Positive,Positive,Positive,Positive,Positive,Neutral,0.034550
4874,Investment Research,IR-4891,BP,2019-02-20,,Positive,Positive,Neutral,Neutral,Neutral,Neutral,0.002359
9849,Earnings Call Presentations,EP-317,BP,2019-04-30,,Positive,Positive,Positive,Neutral,Positive,Neutral,0.016503
10100,Earnings Call Q&A,EQ-317,BP,2019-04-30,,Positive,Positive,Positive,Positive,Positive,Neutral,0.016503


Source             0
Unique_ID          0
Ticker             0
Date               0
URL                0
Finance            0
Production         0
Reserves/M&A       0
Regulatory         0
Green Energy       0
OG Price           0
Returns         3829
dtype: int64


In [11]:
# Check out some rows with NA's
df_with_nas = df_merged[df_merged.isna().any(axis=1)]
df_with_nas['DayOfWeek'] = df_with_nas['Date'].dt.day_name()
display(df_with_nas.head())

<ipython-input-11-1915ee96ceca>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_nas['DayOfWeek'] = df_with_nas['Date'].dt.day_name()


,Source,Unique_ID,Ticker,Date,URL,Finance,Production,Reserves/M&A,Regulatory,Green Energy,OG Price,Returns,DayOfWeek
7376,ProQuest,PQ-2244415497,BP,2019-06-22,https://www.proquest.com/newspapers/bp-plc/doc...,Positive,Positive,Negative,Neutral,Positive,Neutral,NaN,Saturday
8587,SEC Filings,SEC-114036119010687,BP,2019-07-06,https://www.sec.gov/Archives/edgar/data/000031...,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,NaN,Saturday
7373,ProQuest,PQ-2257088783,BP,2019-07-13,https://www.proquest.com/newspapers/best-pensi...,Positive,Neutral,Negative,Neutral,Neutral,Neutral,NaN,Saturday
7374,ProQuest,PQ-2257088132,BP,2019-07-13,https://www.proquest.com/newspapers/best-pensi...,Positive,Neutral,Neutral,Neutral,Neutral,Neutral,NaN,Saturday
7358,ProQuest,PQ-2274069207,BP,2019-08-17,https://www.proquest.com/newspapers/reliance-i...,Positive,Neutral,Positive,Neutral,Neutral,Neutral,NaN,Saturday


In [12]:
# Fill NaN values with the next day's value within the same 'Ticker'
df_merged['Returns'] = df_merged.groupby('Ticker')['Returns'].fillna(method='bfill')
display(df_merged.head())
print(df_merged.isna().sum())

,Source,Unique_ID,Ticker,Date,URL,Finance,Production,Reserves/M&A,Regulatory,Green Energy,OG Price,Returns
9863,Earnings Call Presentations,EP-331,BP,2019-02-05,,Positive,Positive,Positive,Positive,Positive,Neutral,0.034550
10113,Earnings Call Q&A,EQ-331,BP,2019-02-05,,Positive,Positive,Positive,Positive,Positive,Neutral,0.034550
4874,Investment Research,IR-4891,BP,2019-02-20,,Positive,Positive,Neutral,Neutral,Neutral,Neutral,0.002359
9849,Earnings Call Presentations,EP-317,BP,2019-04-30,,Positive,Positive,Positive,Neutral,Positive,Neutral,0.016503
10100,Earnings Call Q&A,EQ-317,BP,2019-04-30,,Positive,Positive,Positive,Positive,Positive,Neutral,0.016503


Source            0
Unique_ID         0
Ticker            0
Date              0
URL               0
Finance           0
Production        0
Reserves/M&A      0
Regulatory        0
Green Energy      0
OG Price          0
Returns         540
dtype: int64


In [13]:
# Drop last Nas
df_merged = df_merged.dropna()
print(df_merged.isna().sum())

Source          0
Unique_ID       0
Ticker          0
Date            0
URL             0
Finance         0
Production      0
Reserves/M&A    0
Regulatory      0
Green Energy    0
OG Price        0
Returns         0
dtype: int64


In [14]:
# save csv locally
df_merged.to_csv('02_Gemini_Prepped_Stock_Data.csv', index=False)